# 🚀人事異動情報を収集するためのツールです

免責事項:
当ツールによって生じた如何なるトラブル・損失・損害に対してもツール作成者は責任を負うものではないことを予めご了承ください。

## ⚙️ パラメータについて
|#|パラメータ名|とりえる値|説明|
|:--|:--:|:--:|:--|
|1|TARGET_DATE|'1'~'31'|収集対象の人事情報の掲載日|
|2|FI_WORDS|list[str]|企業が金融機関かどうかを判断するキーワード。この中のキーワードのいずれかが企業名に含まれていた場合に収集対象とする。|
|3|GRCS_WORDS|list[str]|人事異動情報の詳細を表示するかどうか判断するキーワード。この中のキーワードのいずれかが人事異動情報に含まれていた場合に、結果に企業名だけでなく人事異動情報の詳細を出力する。|

## 📖 使い方
1. パラメータ3つを適切な値に設定する
2. 実行
3. 最下部に結果が表示される

## 🔖 制約
- 当日更新分は取得不可
- 過去1−2週間程度のみ取得可能

---

### パラメータ修正箇所↓

In [ ]:
TARGET_DATE: str = '27'
FI_WORDS = ['銀行', 'バンク', '金庫', '信金', '信用', '組合', 'ファイナンシャル', 'フィナンシャル', 'ファイナンス', '証券', '證券', '保険', '損保', '生命', '生保', '金融庁']
GRCS_WORDS = ['IT', 'システム', 'リスク', '金融犯罪', 'AML', 'コンプライアンス', '犯罪', 'DX', 'デジタル']



---

In [ ]:
import requests
from bs4 import BeautifulSoup
import lxml.html
import time
import datetime
import random

In [ ]:
def is_target_date(target_date: str, content_date: str):
    return '日' in content_date and target_date == content_date[0:len(target_date)]

def is_financial_institution(title: str):
    return any(fi_word in title for fi_word in FI_WORDS)

def is_GRCS_related(article: str):
    return any(grcs_word in article for grcs_word in GRCS_WORDS)


In [ ]:
BASE_URL: str = 'https://www.nikkei.com'
MAX_REQUESTS = 1000

In [ ]:
TITLE_SELECTOR = '#CONTENTS_MAIN > section > ul:nth-of-type({}) > li:nth-of-type({}) > div > div.col.headline > h3 > a'
    ##CONTENTS_MAIN > section > ul:nth-child(2) > li:nth-child(1) > div > div.col.headline > h3 > a
CONTENT_DATE_SELECTOR = '#CONTENTS_MAIN > section > ul:nth-of-type({}) > li:nth-of-type({}) > div > div.col.time > p'
    ##CONTENTS_MAIN > section > ul:nth-child(2) > li:nth-child(1) > div > div.col.time > p
HR_CONTENT_XPATH = '/html/body/div[5]/main/article/section[1]'

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
}
session = requests.Session()

In [ ]:
def main() -> list:
    stage = 1  # stage: 1-TARGET_DATEに達していない状態, 2-達した状態
    result = []
    cnt = 0  # cnt: ループの回数を記録する
    for bn in range(1, 1982, 30):  # ページのURLが /?bn=31, /?bn=61のように30刻みとなっていることに対応
        time.sleep(random.randint(1, 2))
        soup = BeautifulSoup(session.get(BASE_URL+'/news/jinji/hatsurei/?bn='+str(bn), headers=HEADERS).text)
        # print('[{}] Proceeding with Page-{}'.format(datetime.datetime.now(), bn // 30 + 1))
        for ul_num in range(1, 4):
            for li_num in range(1, 11):
                cnt += 1
                if cnt >= MAX_REQUESTS:  # 安全考慮
                    result.append('Too many requests > {}'.format(MAX_REQUESTS))
                    return result
                header = soup.select(TITLE_SELECTOR.format(ul_num, li_num))
                link, title = header[0].attrs['href'], header[0].contents[0].contents[0]

                time_box = soup.select(CONTENT_DATE_SELECTOR.format(ul_num, li_num))
                content_date = time_box[0].contents[0]
                
                print('\r                                                                                 \r', end='')
                print('   [{}] Checking - ({}) {}'.format(cnt, content_date, title), end='', flush=True)

                if is_target_date(TARGET_DATE, content_date):
                    stage = 2
                    if is_financial_institution(title):
                        time.sleep(random.randint(1, 2))
                        html = lxml.html.fromstring(session.get(BASE_URL+link, headers=HEADERS).text)
                        hr_article = html.xpath(HR_CONTENT_XPATH)[0].text_content()
                        if is_GRCS_related(hr_article):
                            result.append('[{}]({}) ({}) > {}'.format(title, BASE_URL + link, content_date, hr_article))
                        else:
                            result.append('[{}]({}) ({}) '.format(title, BASE_URL + link, content_date))

                else:
                    if stage == 2:
                        print('')
                        return result
    print('')
    return result


In [ ]:
print('[{}] Starting crawl_hr_news_in_nikkei for TARGET_DATE = {}'.format(datetime.datetime.now(), TARGET_DATE))
result = main()
print('[{}] Completed crawl_hr_news_in_nikkei for TARGET_DATE = {}. Result follows...'.format(datetime.datetime.now(), TARGET_DATE))
print('-----------------------------------------')
if result:
    for text in result:
        print(text)
else:
    print('No result')
print('-----------------------------------------')

# 以上